# Download all the Cubesat data

Using ONLY sequence numbers because NA is only providing us with a sequence number (yey?)

In [1]:
# Import libraries
import raadpy as rp
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [4]:
# Download based on sequence number
def download_seq_nr(fileName:str,start:int=-1,end:int=None,MAX:int=5000):
    # Input processing
    if end is None: end = float('inf')
    
    # Sequence number and other stuff init
    seq         = start
    host        = "https://light1.mcs.nanoavionics.com"
    url         = f'{host}/{fileName}_download?'
    
    # Initialize array to hold the buffer raw data
    data        = [] 

    # Do unless interrupted
    while seq < end:
        # Find the query for NA Server
        query       = f'seq_nr=gt.{seq}&seq_nr=lte.{seq+MAX}'
        new_data    = rp.download_range(url+query,token=rp.TOKEN,limit=-1)
        
        # Update variables
        data        = data + new_data
        seq         = seq + MAX

        # Exit condition with searching one last time
        if len(new_data) == 0:
            data = rp.sort(data)    # Sort by "entry_nr"
            break

    return data

# Download script packet
def download_data(filepath:str='./',buffers=range(1,8),MAX:int=5000):
    # List that holds all the filenames
    filenames = []

    # First go ahead and download all the buffers
    for buffer in tqdm(buffers,desc='Downloading Buffer'):
        # Download the data of the buffer
        fileName    = "pc_buff"+str(buffer)
        data        = download_seq_nr(fileName,MAX=MAX)

        # Save the data of the buffer
        fname   = rp.save_raw_data(data,filepath=filepath,buffer=buffer)
        filenames.append(fname)

    # Download the script log
    log         = download_seq_nr('pc_se0_log')
    log         = rp.log_to_ascii(log,fileName=filepath+'light1-se-log.txt')
    decoded_log = rp.log_expand(text=log)

    # Extract the metadata from the logfile
    metadata = rp.log_metadata(decoded_log=decoded_log)

    # Save the datafile as a json on the same directory
    with open(filepath + "metadata.json","w") as meta_file: rp.json.dump(metadata,meta_file,indent=4)

    return metadata

In [5]:
download_data()

TypeError: object of type 'bool' has no len()